<center>
    <h1>American Sign Language Alphabet Classification</h1><h5>Wilbert Garcia, Madelyn Gatchel, Nyjay Nelson</h5>
    <img src="asl_alphabet.png" alt="Each letter in ASL alphabet">
</center>


<!-- https://commons.wikimedia.org/wiki/File:American_Sign_Language_Alphabet.gif  -->

### **Problem Explanation**

Sign Language is a form of communication that does not use words but rather uses unique hand gestures to represent letters and words. Deep Learning can be used to classify the different letters of the American Sign Language (ASL) alphabet. This research is important in further developing technology for those with hearing disabilities. 


For our project, we are experimenting with several deep learning transfer learning models available for image classification in order to perform multi-class classification on the dataset. The goal is to train a deep neural network that classifies the dataset and provides a model that accurately predicts the different letters of the sign language alphabet.

### **Related Work**

Through extensive research and reading of related work, it seems that research pertaining to ASL is limited in comparison to other computer vision tasks. ASL, as form of communication, is ["one of the most understudied areas"](https://arxiv.org/pdf/1710.06836.pdf). 


The goal of this assignment is to train a deep network but deep networks are very difficult to trai as they require much more data, computing power and time. We can avoid the issues of training a network from scratch by taking advantage of large neural networks that others have already trained using training learning models. 


We employ three transfer learning models offered by Keras from a collection of deep learning models that are available with pre-trained weights. We use th [VGG16 model](https://keras.io/api/applications/vgg/#vgg16-function), the [InceptionV3](https://keras.io/api/applications/inceptionv3/) model and the [ResNet50](https://keras.io/api/applications/resnet/#resnet50-function) model. Each model is pre-trained using the ImageNet. ImageNet is a fairly comprehensive dataset. It features millions of images and thousands of different classes. An Image classification model that is trained on ImageNet and performs with significant accuracy generalizes well to new data and is not subject to overfitting. The ability to generalize well to new data and the accuracy of these models are attractive features as we use them to attempt to classify the ASL alphabet.

### **Dataset Formulation**

The datasets from this project come from Kaggle. The first dataset is owned by Kuzivakwashe Muvezwa and is referred to as [Significant (ASL) Sign Language Alphabet Dataset](https://www.kaggle.com/kuzivakwashe/significant-asl-sign-language-alphabet-dataset). The second dataset is uploaded and owned by Akash and is called the [ASL Alphabet](https://www.kaggle.com/grassknoted/asl-alphabet).


These datasets were the only two image datasets we could find. The University of Texas and Boston University collaborated to create [video datasets](http://vlm1.uta.edu/~athitsos/asl_lexicon/) of the ASL alphabet. These datasets contained short video clips that could have been useful in creating our dataset but, given the time constraint of the project and the sufficient data that we had already collected, we decided to focus on the loading and preprocessing of the data.


The process to create the dataset is compromised of two steps. The first step is to delete duplicates between the datasets and then the second step is to consolidate the datasets. The purpose of deleting duplicates is to prevent overfitting. The network could potentially overfit to those duplicated images given the higher frequency of certain images in the training dataset. Another reason that duplicate images are discouraged is that duplicate images in the testing set are likely to increase testing accuracy, testing precision and testing recall an amount that is not truly indicative of the accuracy, precision and recall of the model. We outline three notebooks that describe how we determine duplicates in our datasets and proceed to remove said duplicates.
   
   
   * `test_uniqueness.ipynb`: To test the uniqueness of a given image, we loop through each image in one dataset with the same label in the other dataset. We read both of those image names and compare whether the array/grid representations of the images are numerically equal. If they are equal, we print the names of the duplicate images. Through this process, it became clear that this method of testing uniqueness would be unrealistic and infeasible because of the amount of time it takes to run. However, at the same time, it became clear that the majority of duplicate images had the same name in both datasets. Thus, we decided it would be sufficient to remove duplicates where the names of the images are exactly the same.
   
   
   * `remove_duplicates.ipynb`: For each image with label x in the first dataset and each image with label x in the second dataset, we check whether the two images have the same name and if they do, we remove the image from the second dataset. 
 

 * `consolidate_data.ipynb`: All of the remaining images from the second dataset are moved to the first dataset and all of the second dataset directories are deleted because they are empty. 

### **Pre-Processing**

Only one dataset came with a designated testing set. This testing set only included one image per letter/label, so we decided to not use this testing set. Instead, we decided to create our own test set to include images from both datasets. To split the consolidated dataset into a training and test set, we used the [split-folders package](https://pypi.org/project/split-folders/). We used 80% of our data for the training set and the remaining 20% for the test set. This can be seen in `train_test_splitdir.ipynb`. We have provided a reference to the process for context and clarity but this file will not run since the datasets have already been split into training and test datasets.

For data preprocessing, we use the ImageDataGenerator class which generates batches of tensor image data. It takes a data path to a directory & generates batches of augmented data. The ImageDataGenerator class returns a Directory Iterator. which is a directory that is able to read images from a directory. This directory iterator is composed of tuples of (x, y) where x is a numpy array containing a batch of images. The shape of this numpy array is(batch_size, target_size, channels). The y component of the tuple is a numpy array of corresponding labels. 

The ImageDataGenerator class is useful to perform image augmentations, which proved useful in the binary classification of pneumonia using X-ray images. The ImageDataGenerator class augments data performing various transformations to create more training data. We experimented with image transformations but found that our consolidated dataset was sufficiently large that we did not need to perform image transformations.

We used the ImageDataGenerator class using the flow_from_directory function to  separate the data into batches and load each batch. This facilitates the process of loading data eliminating the need to manually load a given batch and then have to delete it from memory.

The ImageDataGenerator class also helped uniformly resize our images.From initial inspection, it seemed that all images had dimensions ranging from 180x180 to 255x255. We decided to give all images the uniform dimension, 200x200. This resulted in images that were smaller being padded and images that were larger were resized to the specified dimension.

In [1]:
import numpy as np
from os import listdir # used to get labels
from tensorflow.keras.preprocessing.image import ImageDataGenerator

The next two cells will look pretty similar to the corresponding cells in our project 5 writeup. It is important to note that the `class_mode` is categorical (not binary) because we are performing multi-class classification. Also informal experimentation showed that varying the batch size and steps per epoch did not seem to have a significant effect on the results so we used `BATCH_SIZE = 32` AND `TRAIN_STEPS = 50` (steps per epoch) for all experiments. Our plan was to go back and vary the batch size and steps per epoch if the other experiments we tried were not successful, but since we were successful (spoiler alert!), we did not end up varying them.

In [2]:
training_path = "asl_data/train"
test_path = "asl_data/test"
labels = listdir(training_path)
NUM_LABELS = len(labels)
BATCH_SIZE = 32
TRAIN_STEPS = 50
VERBOSE=2 # change to 0 for no updates or 1 for full updates
print(labels)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(training_path, target_size=(200, 200), classes = labels, 
                                                    batch_size = BATCH_SIZE, class_mode='categorical', shuffle=True)
test_generator = test_datagen.flow_from_directory(test_path, target_size=(200, 200), classes = labels,
                                                  batch_size = BATCH_SIZE, class_mode='categorical', shuffle=False)
y_test = test_generator.classes
TEST_STEPS = np.math.ceil(test_generator.samples / BATCH_SIZE) # ensures we predict every image in test set exactly once in one epoch

Found 124910 images belonging to 29 classes.
Found 31241 images belonging to 29 classes.


### **Experiments**
#### **Model Architectures** 


  * The models outlined below can be found in `model.py`. This file contains functions to create and compile each model. User can output a model summary by changing `print_summary=False` to `True` in the `get_model()` function call. Since we had success with transfer learning models in project 5 (Binary Classification of Pneumonia using X-ray Images) and this is a harder learning problem, we decided to use transfer learning models for this project as well.
     
    * `baseline_model`: This model contains a singular softmax classification layer and serves as a basline because no other (more complex) model should perform worse than this model. 

    * `inceptionv3_model`: The [InceptionV3](https://arxiv.org/abs/1512.00567) architecture that focuses on optimizing the computation power of the network using factorized convolutions, regularization techniques and aiming for a small number of parameters. The [InceptionV3](https://blog.paperspace.com/popular-deep-learning-architectures-resnet-inceptionv3-squeezenet/) model is a convolutional network that uses factorized convolutions to improve computational efficiency. The architecture uses smaller convolutions that lead to faster training of the model and asymmetric convolutions that result in using less parameters. InceptionV3 uses an auxillary classifier as regularizer. We chose the Inception V3 architecture as one of the transfer learning models in our experiment because it achieved Top-1 Accuracy of 0.779 and Top-5 Accuracy of 0.937 which represent the performance of the model on the ImageNet validation set.
    
    * `resnet50_model`: The [ResNet50](https://arxiv.org/pdf/1512.03385.pdf) architecture is a residual network that allows for a substantially deeper network than other networks. The ResNet50 architecture is made up of 50 layers made up of residual blocks with skip connections that allow inputs to forward propagate faster through the residual connection across layers making the initial training more useful throughout the deep residual network. We chose the ResNet50 architecture because it achieved Top-1 Accuracy of 0.749 and Top-5 Accuracy of 0.921 which represent the performance of the model on the ImageNet validation set. We also found it attractive that while ResNet models are much deeper than VGG models, they are still less complex making for an attractive model.
       
    * `vgg16_model`: The [VGG16](https://arxiv.org/pdf/1409.1556.pdf) [architecture](https://neurohive.io/en/popular-networks/vgg16/) is a convolutional neural network created by the Visual Geometry group at the University of Oxford. The model has 22 layers of five convolutional blocks followed by three fully connected dense layers and an output layer. The first two convolutional blocks are made up of two convolutional layers with ReLu activation and max pooling layer. The three convolutional blocks that follow are made up of three convolutional layers with ReLu activation and max pooling layer. The Dense layers also have ReLu activation and the output layer uses a softmax activation function. We chose the VGG16 architecture as one of the transfer learning models in our experiment because it achieved Top-1 Accuracy of 0.713 and Top-5 Accuracy of 0.901 which represent the performance of the model on the ImageNet validation set.

In the cell below, we create and compile each model using the corresponding function in `model.py.` In terms of architecture, each of the three transfer models includes the transfer model layers, a dense layer with ReLU activation, a dropout layer, and the softmax classification layer. Note that since we are using softmax activation for multi-class classification, we use categorical cross entropy for our loss function. For now, you can ignore the other keyword arguments (they are the optimal hyperparameter settings based on the experiments we ran).

In [4]:
import model
baseline_model = model.get_baseline_model(print_summary=False);
inceptionv3_model = model.get_inception_model(trainable=True, num_nodes=512, learning_rate=.0001, print_summary=False);
resnet50_model = model.get_resnet50_model(trainable=True, print_summary=False);
vgg16_model = model.get_vgg16_model(trainable=False, num_nodes=256, learning_rate=.001, print_summary=False);

#### **Metrics** 
Because implications of false negatives and false positives are much greater in communication context, we look at several metrics other than just binary accuracy to determine the best model:

* Precision: $\frac{TP}{TP+FP}$
* Recall: $\frac{TP}{TP+FN}$
* F1 score: $\frac{2\cdot precision * recall}{precision + recall}$

Since we are doing multi-class classification, the overall [precision, recall, f1] score is the weighted [precision, recall, f1] score from each class as calculated by `sklearn`. Note that we use binary acccuracy instead of categorical accuracy because our predictions are not one-hot encoded. Finally, we have written the `print_results()` function in `model.py` to print the prediction, binary accuracy, precision, recall, f1 score and confusion matrix.

#### **Experimental Specification**

In our experiments, we vary the following hyperparameters: 

| num_dense_nodes | dropout_percentage | learning_rate | unfreeze_weights |  
|------|----|----|------|
| 64, 256, 512| .1, .25 | .001, .0001 | False, True |

Note that `unfreeze_weights` refers to whether we allow the network to train the transfer model weights or not. In this case, True means yes, you are able to train the weights. There are obviously many more combinations of hyperparameter settings that we could have tried. But, given the time constraints of the project, we used to prior knowledge and experience to choose combinations that we believed would allow us to find an optimal model.

For each experiment and each model, we trained 5 epochs at a time until the loss stopped decreasing and/or the accuracy and or f1 scores decreased. Then, we recorded the previous number of epochs. Our results can be found in `experimental_results.ipynb`. 

We now show how the optimized models which have the best hyperparameter settings for each model compare against each other. NOTE: Since we could not get ResNet to work, we will not be training Resnet here.

In [5]:
vgg16_model.fit(train_generator, batch_size=BATCH_SIZE, steps_per_epoch=TRAIN_STEPS, epochs= 15, verbose=VERBOSE)
predictions = vgg16_model.predict(test_generator, verbose=1, steps=TEST_STEPS)
accuracy, precision, recall, f1, conf_mat = model.print_results(y_test, predictions)

Epoch 1/15
50/50 - 4s - loss: 3.3337 - binary_accuracy: 0.9652 - precision: 0.2308 - recall: 0.0037
Epoch 2/15
50/50 - 4s - loss: 2.4523 - binary_accuracy: 0.9669 - precision: 0.8404 - recall: 0.0494
Epoch 3/15
50/50 - 4s - loss: 1.9456 - binary_accuracy: 0.9702 - precision: 0.9222 - recall: 0.1481
Epoch 4/15
50/50 - 4s - loss: 1.6280 - binary_accuracy: 0.9728 - precision: 0.8910 - recall: 0.2400
Epoch 5/15
50/50 - 4s - loss: 1.3457 - binary_accuracy: 0.9770 - precision: 0.9211 - recall: 0.3650
Epoch 6/15
50/50 - 4s - loss: 1.2076 - binary_accuracy: 0.9781 - precision: 0.8886 - recall: 0.4187
Epoch 7/15
50/50 - 4s - loss: 1.1021 - binary_accuracy: 0.9793 - precision: 0.8875 - recall: 0.4588
Epoch 8/15
50/50 - 4s - loss: 0.9908 - binary_accuracy: 0.9814 - precision: 0.8926 - recall: 0.5244
Epoch 9/15
50/50 - 4s - loss: 0.8717 - binary_accuracy: 0.9833 - precision: 0.9103 - recall: 0.5706
Epoch 10/15
50/50 - 4s - loss: 0.9023 - binary_accuracy: 0.9833 - precision: 0.8972 - recall: 0.5838

In [6]:
inceptionv3_model.fit(train_generator, batch_size=BATCH_SIZE, steps_per_epoch=TRAIN_STEPS, epochs= 10, verbose=VERBOSE)
predictions = inceptionv3_model.predict(test_generator, verbose=1, steps=TEST_STEPS)
accuracy, precision, recall, f1, conf_mat = model.print_results(y_test, predictions, class_report=True)

Epoch 1/10
50/50 - 6s - loss: 2.8966 - binary_accuracy: 0.9769 - precision: 0.8880 - recall: 0.3766
Epoch 2/10
50/50 - 6s - loss: 1.4215 - binary_accuracy: 0.9764 - precision: 0.8024 - recall: 0.4187
Epoch 3/10
50/50 - 6s - loss: 0.7489 - binary_accuracy: 0.9859 - precision: 0.8687 - recall: 0.6950
Epoch 4/10
50/50 - 6s - loss: 0.5646 - binary_accuracy: 0.9889 - precision: 0.8849 - recall: 0.7781
Epoch 5/10
50/50 - 6s - loss: 0.3680 - binary_accuracy: 0.9920 - precision: 0.9127 - recall: 0.8494
Epoch 6/10
50/50 - 6s - loss: 0.3602 - binary_accuracy: 0.9923 - precision: 0.9134 - recall: 0.8569
Epoch 7/10
50/50 - 6s - loss: 0.2593 - binary_accuracy: 0.9942 - precision: 0.9348 - recall: 0.8956
Epoch 8/10
50/50 - 6s - loss: 0.2778 - binary_accuracy: 0.9945 - precision: 0.9404 - recall: 0.8975
Epoch 9/10
50/50 - 6s - loss: 0.2367 - binary_accuracy: 0.9953 - precision: 0.9499 - recall: 0.9131
Epoch 10/10
50/50 - 6s - loss: 0.1997 - binary_accuracy: 0.9961 - precision: 0.9563 - recall: 0.9300

### **Results and Conclusions**

Based on our experiments, the best model for classifying the ASL alphabet is the InceptionV3 model with 512 dense nodes, a dropout rate of 0.1  and  having unfreeze_weights set to equal True. In training and testing this model several times with these settings, we always had a test binary accuracy ranging from .92 to .95 and f1 score ranging from .92 to .95. Given that this problem is a much harder learning problem and these results are better than results on previous homeworks, we are happy with our results. That being said, we acknowledge that there are likely other hyperparameter settings for InceptionV3 and possibly for VGG16 that produce similar or even better results. 

Given the opportunity to work as group on research that combines technology and allows us to provide a social good, we have thoroughly enjoyed applying our passion of Deep Learning to a relevant challenge. 

#### **Reflections and Considerations for Future Exploration:**

One criticism of the two datasets we used is that the images contain the hands of primarily contain images of white male hands. This issue of misrepresented data can lead to biases as early as the data collection process. Leading to a harmful cascading effect that can result in ethical questions relating to oppressive software and algorithms.

An avenue for further exploration is that because there are fewer images for J and Z classes compared to the other classes, image transformations for these two classes to have balanced classes within our training set. The imbalance in the number of images for those two letters may be due to the fact that they are the two letters is the ASL alphabet that are dynamic in their representation, requiring movement in comparison to the rest of the alphabet which is static in its representation. However, as the classification report shows, right now our model does pretty well at predicting letters J and Z.

Also, we are still confused as to why we could not get ResNet50 to work at all, so with additional time it would be interesting to see if we could get it working. 

### **References**
[1] https://www.kaggle.com/kuzivakwashe/significant-asl-sign-language-alphabet-dataset (our first ASL alphabet dataset)<br>
[2] https://www.kaggle.com/grassknoted/asl-alphabet (our second ASL alphabet dataset) <br>
[3] https://arxiv.org/pdf/1710.06836.pdf (this was one of the only papers we found that discussed learning the ASL alphabet; it provided a helpful background and also further supported why this learning problem is still so relevant)<br>
[4] http://vlm1.uta.edu/~athitsos/asl_lexicon/ (American Sign Language Lexicon Video Dataset; we considered trying to extract images from this video dataset but because we already had enough data and because of the time constraints, we did not end up using this dataset) <br>
[5] https://pypi.org/project/split-folders/ (this package helped us split the dataset into a training set and a testing set)<br>
[6] https://arxiv.org/abs/1512.00567 (this paper describes the inception model that we used as a part of our transfer learning model; we didn't read the details of this paper too closely because they weren't necessary for this project, but it was interesting to learn more about the model) <br>
[7] https://blog.paperspace.com/popular-deep-learning-architectures-resnet-inceptionv3-squeezenet/ (brief introduction to several transfer learning models including resnet and inception)<br>
[8] https://arxiv.org/pdf/1512.03385.pdf (this paper describes the resnet model that we used as a part of our transfer learning model that we tried to get working; we didn't read the details of this paper too closely because they weren't necessary for this project, but it was interesting to learn more about the model) <br>
[9] https://arxiv.org/pdf/1409.1556.pdf (this paper describes the vgg model that we used as a part of our transfer learning model; we didn't read the details of this paper too closely because they weren't necessary for this project, but it was interesting to learn more about the model)<br>
[10] https://neurohive.io/en/popular-networks/vgg16/ (this article provides a brief introduction to vgg16 and was helpful to get the most relevant and important details for the model)<br>

